In [1]:
import os
import re
from dataclasses import dataclass, field
from datetime import datetime
from typing import Optional
from PIL import Image
from datasets import load_dataset
from transformers import Qwen2VLForConditionalGeneration

# from math_verify import parse, verify
from trl import GRPOConfig, GRPOTrainer, ModelConfig, ScriptArguments, TrlParser, get_peft_config

.4'
/data/miniconda3/envs/openmllm/compiler_compat/ld: /usr/local/cuda-12.2/lib64/libcufile.so: undefined reference to `std::basic_ifstream<char, std::char_traits<char> >::~basic_ifstream()@GLIBCXX_3.4'
/data/miniconda3/envs/openmllm/compiler_compat/ld: /usr/local/cuda-12.2/lib64/libcufile.so: undefined reference to `std::ios_base::Init::Init()@GLIBCXX_3.4'
/data/miniconda3/envs/openmllm/compiler_compat/ld: /usr/local/cuda-12.2/lib64/libcufile.so: undefined reference to `std::condition_variable::condition_variable()@GLIBCXX_3.4.11'
/data/miniconda3/envs/openmllm/compiler_compat/ld: /usr/local/cuda-12.2/lib64/libcufile.so: undefined reference to `std::basic_filebuf<char, std::char_traits<char> >::basic_filebuf()@GLIBCXX_3.4'
/data/miniconda3/envs/openmllm/compiler_compat/ld: /usr/local/cuda-12.2/lib64/libcufile.so: undefined reference to `VTT for std::basic_istringstream<char, std::char_traits<char>, std::allocator<char> >@GLIBCXX_3.4'
/data/miniconda3/envs/openmllm/compiler_compat/ld: 

In [2]:
SYSTEM_PROMPT = (
    "A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant "
    "first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning "
    "process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., "
    "<think> reasoning process here </think><answer> answer here </answer>"
)
QUESTION_TEMPLATE = " {Question} \n Output the thinking process in <think> </think> and final answer (number) in <answer> </answer> tags."
def make_conversation_image(example):
        return {
            "prompt": [
                {
                    "role": "user",
                    "content": [
                        {"type": "image"},
                        {"type": "text", "text": QUESTION_TEMPLATE.format(Question=example["problem"])},
                    ],
                },
            ],
            "image": Image.open(example['image_urls']) 
        }

In [3]:
dataset = load_dataset('json', data_files='/group/40064/johnbli/Data/GMNER/data/twitter2015/test.json')

In [4]:
dataset['train'][0]

{'image_urls': '/group/40064/johnbli/Data/GMNER/IJCAI2019_data/twitter2015_images/62654.jpg',
 'problem': 'The content in the picture is auxiliary information. Please analyze the content in the picture and combine it with the text information to list all entity words in the text that match the category, and return your results in json format, output format is "\\{word1: type1, word2: type2\\}". The extracted entities must appear in the text, and the images are only auxiliary information. \\nEntity category: location, person, organization, other \nText:RT @ThePatriot143 : TIMELINE SHOWS HOW CLINTONS TOOK $ 1 . 8 MILLION FROM KEYSTONE PIPELINE INVESTORS http://t.co/17CGxAS18d #ClintonCash http:…',
 'solution': '{"CLINTONS": "person", "KEYSTONE": "other"}'}

In [5]:
dataset = dataset.map(make_conversation_image)

Map: 100%|██████████| 3257/3257 [00:48<00:00, 67.38 examples/s]


In [18]:
dataset['train'][0]

{'image_urls': '/group/40064/johnbli/Data/GMNER/IJCAI2019_data/twitter2015_images/62654.jpg',
 'problem': 'The content in the picture is auxiliary information. Please analyze the content in the picture and combine it with the text information to list all entity words in the text that match the category, and return your results in json format, output format is "\\{word1: type1, word2: type2\\}". The extracted entities must appear in the text, and the images are only auxiliary information. \\nEntity category: location, person, organization, other \nText:RT @ThePatriot143 : TIMELINE SHOWS HOW CLINTONS TOOK $ 1 . 8 MILLION FROM KEYSTONE PIPELINE INVESTORS http://t.co/17CGxAS18d #ClintonCash http:…',
 'solution': '{"CLINTONS": "person", "KEYSTONE": "other"}',
 'prompt': [{'content': [{'text': None, 'type': 'image'},
    {'text': ' The content in the picture is auxiliary information. Please analyze the content in the picture and combine it with the text information to list all entity words i

In [7]:
model_path = "/group/40064/johnbli/Code/Deepseek/open-r1-multimodal/checkpoints/Qwen-VL-2B-GRPO-2025-04-18-20-06-29"

In [8]:
import torch
model = Qwen2VLForConditionalGeneration.from_pretrained(model_path, device_map='cuda:0', trust_remote_code=True)#, torch_dtype=torch.float16)

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.28it/s]


In [9]:
from transformers import AutoProcessor
processing_class = AutoProcessor.from_pretrained(model_path)
pad_token_id = processing_class.tokenizer.pad_token_id
processing_class.pad_token_id = pad_token_id
processing_class.eos_token_id = processing_class.tokenizer.eos_token_id

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [10]:
from trl.data_utils import apply_chat_template, is_conversational, maybe_apply_chat_template

prompts = dataset['train'][0]
images = dataset['train'][0]['image']
prompts_text = apply_chat_template(prompts, processing_class)["prompt"]

In [11]:
prompts_text

'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|> The content in the picture is auxiliary information. Please analyze the content in the picture and combine it with the text information to list all entity words in the text that match the category, and return your results in json format, output format is "\\{word1: type1, word2: type2\\}". The extracted entities must appear in the text, and the images are only auxiliary information. \\nEntity category: location, person, organization, other \nText:RT @ThePatriot143 : TIMELINE SHOWS HOW CLINTONS TOOK $ 1 . 8 MILLION FROM KEYSTONE PIPELINE INVESTORS http://t.co/17CGxAS18d #ClintonCash http:… \n Output the thinking process in <think> </think> and final answer (number) in <answer> </answer> tags.<|im_end|>\n<|im_start|>assistant\n'

In [12]:
prompt_inputs = processing_class(
            text=prompts_text,
            images=images,
            return_tensors="pt",
            padding=True,
            padding_side="left",
            add_special_tokens=False,
        ).to(model.device)

In [13]:
print("input_ids: {}".format(prompt_inputs['input_ids'].shape))

input_ids: torch.Size([1, 790])


In [14]:
prompt_inputs['attention_mask'].shape

torch.Size([1, 790])

In [15]:
generated_ids = model.generate(
            **prompt_inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.9,
            top_p = 0,
            top_k = 1
        )
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(prompt_inputs.input_ids, generated_ids)
]

response = processing_class.batch_decode(generated_ids, skip_special_tokens=False)[0]

In [16]:
response

'<think> The text contains the following entities: CLINTONS, KEYSTONE PIPELINE. </think>\n<answer>{"CLINTONS": "person", "KEYSTONE": "location"} </answer><|im_end|>'

In [17]:
answer

NameError: name 'answer' is not defined

In [40]:
sol = parse("D",)

In [41]:
sol

[]

In [42]:
verify(sol,answer)

False

In [4]:
import re

# 示例文本
text = '''```json
{
  "other": "CorcAP"
}
```'''

# 正则表达式
pattern = r'\{[^{}]*\}'

# 搜索匹配项
matches = re.findall(pattern, text, re.DOTALL)
if matches:
    for match in matches:
        print("Found JSON:", match)
else:
    print("No JSON found.")

Found JSON: {
  "other": "CorcAP"
}
